<h1>Train Neural Network To Estimate Average Strenghts and Variations on the Turn</h1>
There was an error in the evaluator for average hand strengths in the PokerAI-Agent. We must therefore retrain the mode. The error came about in generating the training data. 
Beyond correcting the error, we want to change the outputs of the model. We want to leave the outputs at 3, but instead of outputting average win and average loss, which are actually identical, we will generate two estimates for deviations - a std for win/loss and a std for split. 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K
dataFolder = 'simulatedHands/'
modelsFolder = 'trainedModels/strengthEvaluator/'

2022-01-13 19:48:41.255003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 19:48:41.256195: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<h2>Simulated boards</h2>
We can still use 160 thousand simulated hands form the simHandsTurn-Data Set. The error was not to be found in the estimates for Hero-wins and losses. The data frame consists of hero hands and boards, as well as probabilities of winning, loosing, or splitting the hand against random opponent hands.

In [2]:
simHandsTurn = pd.read_csv(dataFolder+'simHandsTurn.csv')
simHandsTurn.shape

(160000, 15)

In [3]:
simHandsTurn.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit
0,5,1,13,3,13,0,10,1,10,3,8,1,0.786286,0.160857,0.052857
1,8,0,12,2,2,2,10,0,11,2,4,1,0.319429,0.659429,0.021143
2,8,2,13,0,9,2,12,0,10,1,11,0,0.799429,0.040000,0.160571
3,2,0,13,0,12,3,12,0,7,3,13,1,0.754857,0.162571,0.082571
4,5,0,10,3,8,0,11,1,14,0,10,2,0.532000,0.403429,0.064571


Let's check how accurate those percentages are, running a few of those lines against all possible villains.

In [3]:
from itertools import combinations
from PokerHandStrengths import compare
#create a deck
deck=[]
for val in range(2,15):
    for suit in range(4):
        deck.append((val,suit))


In [4]:
#function that takes in a row from the dataFrame and creates the list for hands usable in the compare function
def transformHand(row):
    rowInt = row.astype(int)
    hero = [(rowInt[0], rowInt[1]), (rowInt[2],rowInt[3])]
    board = [(rowInt[4], rowInt[5]), (rowInt[6],rowInt[7]),(rowInt[8], rowInt[9]), (rowInt[10],rowInt[11])]
    return hero + board

In [5]:
transformHand(simHandsTurn.iloc[0])

[(5, 1), (13, 3), (13, 0), (10, 1), (10, 3), (8, 1)]

Let's simulate nwin, nloose, and nsplit

In [6]:
def simTurn(hand, nsim=2000,deck = deck):
    import random
    from PokerHandStrengths import compare
    nwin = 0
    nloose = 0
    nsplit = 0
    deckRemaining = set(deck)-set(hand)
    for i in range(nsim):

        sample = random.sample(deckRemaining,3)
        vilHand = sample[:2]+hand[2:]+[sample[2] ]
        heroHand = hand + [sample[2]]
        result = compare({0:heroHand, 1: vilHand})
        if result[0]==result[1]:
            nsplit += 1
        elif result[0]<result[1]:
            nwin += 1
        else:
            nloose+=1
    return nwin/nsim, nloose/nsim, nsplit/nsim

In [7]:
nwin = []
nloose = []
nsplit = []
for i in range(10):
    handSim = transformHand(simHandsTurn.iloc[i])
    result = simTurn(handSim, nsim=20000)
    nwin.append(result[0])
    nloose.append(result[1])
    nsplit.append(result[2])


In [8]:
#save in numpy array
simResults = np.array([nwin, nloose, nsplit]).transpose()
simResults

array([[0.7882 , 0.15325, 0.05855],
       [0.3437 , 0.639  , 0.0173 ],
       [0.79995, 0.03835, 0.1617 ],
       [0.79095, 0.1546 , 0.05445],
       [0.56375, 0.39255, 0.0437 ],
       [0.797  , 0.04055, 0.16245],
       [0.0947 , 0.887  , 0.0183 ],
       [0.11775, 0.60445, 0.2778 ],
       [0.7296 , 0.2288 , 0.0416 ],
       [0.4581 , 0.4299 , 0.112  ]])

Let's check how well the percentages in the dataFrame simHandsRiver fit the newly simulated ones the exact ones, or simulated ones.

In [9]:
((simHandsTurn.iloc[0:10,-3:] -simResults)**2).mean()

nwin      0.000394
nloose    0.000193
nsplit    0.000147
dtype: float64

We see that there are significant deviations. Let's resimulate the results in the data frame.

In [10]:
#create a cluster of threads and start
nThreads = 8
import ipyparallel as ipp
cluster = ipp.Cluster(n=nThreads)
await cluster.start_cluster()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1641635502-xs5u', profile='default', controller=<running>, engine_sets=['1641635503'])>

In [11]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|█████████████████████████████████████████| 8/8 [00:04<00:00,  1.74engine/s]


In [138]:
i=7
step = 20000
simHandsTurn.iloc[i*step:i*step+step]

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit
140000,9,1,12,2,2,1,9,3,8,1,4,1,0.782857,0.210571,0.006571
140001,6,2,12,0,5,0,2,2,7,0,11,1,0.249429,0.738286,0.012286
140002,5,2,8,2,5,3,8,1,6,2,8,0,0.969429,0.013714,0.016857
140003,8,2,10,3,2,3,8,1,14,3,12,2,0.619143,0.350000,0.030857
140004,2,1,6,3,11,1,4,1,10,3,13,3,0.080286,0.870000,0.049714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,2,3,4,2,12,1,13,3,10,0,7,1,0.048286,0.843714,0.108000
159996,7,1,11,0,12,0,10,0,4,2,10,3,0.291429,0.643143,0.065429
159997,13,1,14,2,12,1,10,3,12,2,7,0,0.602857,0.372000,0.025143
159998,3,3,6,0,9,3,9,0,2,0,5,3,0.180286,0.798000,0.021714


In [20]:
simHandsTurnNew = simHandsTurn.copy()
step = 20000
for i in range(8):
    rc[i]['data'] = simHandsTurn.iloc[i*step:i*step+step]
rc[:]['simTurn']=simTurn
rc[:]['transformHand']= transformHand
rc[:]['step']=step

def simTurnMulti(instance):
    import numpy as np
    nwin = []
    nloose = []
    nsplit = []
    for i in range(step):
        handSim = transformHand(data.iloc[i])
        result = simTurn(handSim, nsim=2000)
        nwin.append(result[0])
        nloose.append(result[1])
        nsplit.append(result[2])
    return np.array([nwin, nloose, nsplit]).transpose()

In [21]:
winList = rc[:].map_sync(simTurnMulti,range(8))
arr = winList[0]
for i in range(1,8):
    arr = np.append(arr, winList[i], axis = 0)
simHandsTurnNew.iloc[:,-3:]=arr
simHandsTurnNew.to_csv(dataFolder+'simHandsTurnNew.csv',index = None)

Let's check the differences of this new data set to the simulated ones from before again and see how much we improved.

In [40]:
((simHandsTurnNew.iloc[:10,-3:]-simResults)**2).mean()

nwin      0.000061
nloose    0.000047
nsplit    0.000036
dtype: float64

The error improved by an order of magnitude. Now it's time to train a model.

In [8]:
from sklearn.model_selection import train_test_split
simHandsTurnNew = pd.read_csv(dataFolder+'simHandsTurnNew.csv')
train, test = train_test_split(simHandsTurnNew, test_size=0.05)
simHandsTurnNew.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit
0,5,1,13,3,13,0,10,1,10,3,8,1,0.7920,0.1465,0.0615
1,8,0,12,2,2,2,10,0,11,2,4,1,0.3470,0.6355,0.0175
2,8,2,13,0,9,2,12,0,10,1,11,0,0.8085,0.0340,0.1575
3,2,0,13,0,12,3,12,0,7,3,13,1,0.8005,0.1510,0.0485
4,5,0,10,3,8,0,11,1,14,0,10,2,0.5615,0.4010,0.0375


In [9]:
X_train = np.array(train.iloc[:,0:-3])
X_test = np.array(test.iloc[:,0:-3])
y_train = np.array(train.iloc[:,-3:])
y_test = np.array(test.iloc[:,-3:])

Let's generate a model.

In [10]:
input_dims = 12

fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4

lr = 0.0001
modelTurnNew = keras.Sequential()
modelTurnNew.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(fc7_dims, activation = 'relu'))
modelTurnNew.add(keras.layers.Dense(3,activation='softmax')) #pwin, ploose and pplit must add up to 1.
modelTurnNew.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

Train the Model. Our goal is a loss below .001

In [ ]:
from keras import backend as K
lr = 0.00001
model = modelTurnNew
#K.set_value(model.optimizer.learning_rate, lr)
model.fit(X_train, y_train, batch_size=8, epochs=20)


In [16]:
modelTurnLarge.save(modelsFolder+'StrengthTurnCorrectedLarge.h5')

In [13]:
modelTurnNew.evaluate(X_test, y_test)

250/250 [==============================] - 1s 2ms/step - loss: 0.0169


0.01691749505698681

In [153]:
prediction[:,1].mean()

0.47252667

In [16]:
lst=[[1,2],[3,4], [5,6],[7,8]]
pd.DataFrame(columns= ['a','b'], data=np.array(lst))

,a,b
0,1,2
1,3,4
2,5,6
3,7,8


<h2>Simulating average hand strengths for training data </h2>
Now that we have a propper network to evaluate individual hand strengths, let's generate training data for average hand strengths. Instead of providing one single hand for hero, we will use all possible combinations, estimate win and loose percentages. Win and loose should be equal. This will also be our sanity check. Because the win and loose probabilities are estimated by a network, they will have some error. So we will use the mean of both as estimator for the strength of average hands. Win-loose- and split percenteges are euqivalent, because they can be calculated from each other. 
Beyond the avg-handStrength, we are interested in a variance meassure of hand strengths. So we will calculate standard deviations for win, and split-pots. The output will therefore be an average value for win/loos, an std for win/loss and a std for splits.

In [11]:
#Function takes in a row from the data frame and calculates pwin/ploose, stdwin, stdloose

def strengthVillain(row):
    import numpy as np
    heroHand = transformHand(row)
    deckRemaining = set(deck) -set(heroHand)
    handsVillain= []
    for combo in combinations(deckRemaining, 2):
        villainHole = list(combo)
        villainHand = np.array(villainHole + heroHand[2:])
        handsVillain.append(list(villainHand.flatten()))
    lstResults=[]
    for i in range(len(handsVillain)):

        rowNew = np.array(handsVillain[i])
        newHand = transformHand(rowNew)
        percentages = playHands(handHero = newHand, deck = deckRemaining)
        percentagesArray = np.array(percentages)

        lstResults.append(percentagesArray)
    pwin = np.array(lstResults)[:,0].mean()
    ploose = np.array(lstResults)[:,1].mean()
    psplit = np.array(lstResults)[:,2].mean()
    stdwin = np.array(lstResults)[:,0].std()
    stdsplit = np.array(lstResults)[:,2].std()
    return pwin, ploose, psplit, stdwin, stdsplit
        

To increase speed, we can calculate handstrengths on multiple threads simulataneously.

In [12]:
#create a cluster of threads and start
nThreads = 8
import ipyparallel as ipp
cluster = ipp.Cluster(n=nThreads)
await cluster.start_cluster()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1641574317-it76', profile='default', controller=<running>, engine_sets=['1641574318'])>

In [13]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|█████████████████████████████████████████| 8/8 [00:07<00:00,  1.03engine/s]


In [14]:


#to use the map function, we need a function to run, with the number of instances/ threads as argument.
def simulateHands(instance):
    import numpy as np
    resultList = []
    for i in range(len(data)):
        results = strengthVillain(data.iloc[i])
        resultList.append(np.array(results))
    return resultList

In [ ]:
startIndex = 32
nThreads = 8
rowsPerThread = 20
for i in range(nThreads):
    #each thread has to recreate the used objects. Let's load all that we need into the threads.
    rc[i]['data'] = simHandsRiver.iloc[startIndex:].iloc[i*rowsPerThread:i*rowsPerThread+rowsPerThread]
    rc[i]['strengthVillain']=strengthVillain
    rc[i]['transformHand']=transformHand
    rc[i]['playHands'] = playHands
    rc[i]['deck'] = deck
    rc[i]['combinations'] = combinations
    rc[i]['compare']=compare
results = rc[:].map_sync(simulateHands,range(nThreads))
strengthVil = pd.DataFrame(columns = ['pwin', 'ploose', 'psplit', 'stdwin', 'stdloose'])
for i in range(len(results)):
    strengthVil = strengthVil.append(pd.DataFrame(results[i], columns=strengthVil.columns), ignore_index=True)
filename = dataFolder+'VillainStrengthCorrected'+ str(startIndex + len(strengthVil))+'.csv'
strengthVil.to_csv(filename, index= None)
pd.read_csv(filename)
                   

In [29]:
strengthVil = pd.DataFrame(columns = ['pwin', 'ploose', 'psplit', 'stdwin', 'stdloose'])
for i in range(len(results)):
    strengthVil = strengthVil.append(pd.DataFrame(results[i], columns=strengthVil.columns), ignore_index=True)
filename = dataFolder+'VillainStrengthCorrected'+ str(len(strengthVil))+'.csv'
strengthVil.to_csv(filename, index= None)
pd.read_csv(filename)
                   

,pwin,ploose,psplit,stdwin,stdloose
0,0.445246,0.445246,0.021628,0.275155,0.016191
1,0.447849,0.447849,0.016423,0.272910,0.013184
2,0.404062,0.404062,0.103998,0.308995,0.094383
3,0.445872,0.445872,0.020378,0.273302,0.015556
4,0.436589,0.436589,0.038943,0.283147,0.033205
5,0.445100,0.445100,0.021920,0.274837,0.017700
6,0.450698,0.450698,0.010725,0.271046,0.013543
7,0.404430,0.404430,0.103261,0.310291,0.095763
8,0.435896,0.435896,0.040329,0.273899,0.024227
9,0.441831,0.441831,0.028458,0.276170,0.019636


In [17]:
input_dims = 14
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4



lr = 0.001
modelRiver = keras.Sequential()
modelRiver.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc7_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))


modelRiver.add(keras.layers.Lambda(lambda x: K.sigmoid(0.1*x)))
modelRiver.add(keras.layers.Dense(2,activation=None))
modelRiver.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')